In [25]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [32]:
train_data_directory = "E:\\研究生\\课程\\我的\\猫狗识别\\train"

In [33]:
#image datapreprocessing
image_size = (224, 224)

batch_size = 32
class_mode = "binary"

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size = image_size,
    batch_size= batch_size,
    class_mode = class_mode,
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size = image_size,
    batch_size= batch_size,
    class_mode = class_mode,
    subset="validation"
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [28]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [29]:
VGG_16 = Sequential()
VGG_16.add(Conv2D(input_shape=(224, 224, 3),filters=64, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
VGG_16.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

VGG_16.add(Flatten())

VGG_16.add(Dense(4096, activation="relu"))
VGG_16.add(Dense(4096, activation="relu"))
VGG_16.add(Dense(1, activation="sigmoid"))

VGG_16.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 256)      

In [30]:
# compile model
from tensorflow.keras import optimizers

VGG_16.compile(loss = "binary_crossentropy",
               optimizer = optimizers.Adam(lr=0.0001),
               metrics=["acc"])

In [31]:
#callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("VGG_16_dog_cat.h5",
                             monitor="val_acc",
                             save_best_only=True,
                             save_weights_only=False,
                             mode="auto",
                             period=1)
early = EarlyStopping(monitor="val_acc",
                      min_delta=0,
                      patience=10,
                      verbose=1,
                      mode="auto")
learning_rate_reduction = ReduceLROnPlateau(monitor="val_acc",
                                            patience=2,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.000001)

In [35]:
nb_epochs = 10
history = VGG_16.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint, early, learning_rate_reduction]
)

ValueError: Asked to retrieve element 0, but the Sequence has length 0